Pretrained Neural Networks like VGG16/VGG19/ResNet/DenseNet are trained on ImageNet which contains 1000-class images. This competition just contains two classes: cat and dog. In this kernel, I want to demonstrate how to build a model with **Pytorch** to classify dog or cat to **Finetuning the convnet**, and then **fix ConvNet to extract image features**. 

This include four steps:

- Build Dog/Cat classify dataset for supervised training.
- Prepare dataset for Pytorch.
- Fintune pretrained ResNet-18 model.
- Fixed ConvNet to extract image features.

Reference:

- [Extract Image features from pretrained NN](https://www.kaggle.com/christofhenkel/extract-image-features-from-pretrained-nn)
- [Transfer Learning Using Pytorch](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html)

**Please UPVOTE if you find it useful** :)

In [2]:
import os
from PIL import Image
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torch.autograd import Variable
from torchvision import datasets, models, transforms
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import time
import copy
plt.ion()   # interactive mode

import time
from tqdm import tqdm, trange
tqdm.pandas()

import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import os
from shutil import copyfile
print(os.listdir(r"./input"))

['breed_labels.csv', 'catdog_folders', 'color_labels.csv', 'densenet-pretrain-with-internet-blocked-12b8d1', 'fatsttext-common-crawl', 'images256', 'keras-applications-weights', 'petfinder-adoption-prediction', 'pytorch-catdog', 'state_labels.csv', 'test', 'train', 'word2vec-google']


In [3]:
train_df = pd.read_csv(r'./input/petfinder-adoption-prediction/train/train.csv')
test_df = pd.read_csv(r'./input/petfinder-adoption-prediction/test/test.csv')
test_df['AdoptionSpeed'] = [-1] * len(test_df)
data_df = pd.concat([train_df, test_df], axis=0).reset_index()
print(train_df.shape[0], test_df.shape[0], data_df.shape[0])

14993 3948 18941


In [4]:
root_dir=r'./input/petfinder-adoption-prediction/train_images/'
save_dir = r'./input/catdog_folders/'
petids = data_df['PetID'].values

In [5]:
model_ft = torch.load('resnet18_catdog_1ep.pth')

In [6]:
def preprocess_image(img):
	img = np.float32(cv2.resize(img, (224, 224))) / 255
	means=[0.485, 0.456, 0.406]
	stds=[0.229, 0.224, 0.225]

	preprocessed_img = img.copy()[: , :, ::-1]
	for i in range(3):
		preprocessed_img[:, :, i] = preprocessed_img[:, :, i] - means[i]
		preprocessed_img[:, :, i] = preprocessed_img[:, :, i] / stds[i]
	preprocessed_img = \
		np.ascontiguousarray(np.transpose(preprocessed_img, (2, 0, 1)))
	preprocessed_img = torch.from_numpy(preprocessed_img)
	preprocessed_img.unsqueeze_(0)
	input = Variable(preprocessed_img, requires_grad = True)
	return input.cuda()
net = torch.load('resnet18_catdog_1ep.pth')
net.eval()

In [7]:

# 000a290e4-1 dog
# 00a1f270a-1 cat
img = cv2.imread('input/petfinder-adoption-prediction/train_images/00a1f270a-1.jpg', 1)
#print(img)
#img = np.float32(cv2.resize(img, (224, 224))) / 255
inp = preprocess_image(img)
#tryim = load_image('input/petfinder-adoption-prediction/train_images/','86e1089a3')
net = model_ft#torch.load('resnet50_trained3.pth')
#print(net)

net.eval()
output = net(inp)
print(output) #print output from crossentropy score

sm = torch.nn.Softmax()
probabilities = sm(output) 
print(probabilities) #Converted to probabilities #tensor[catness, dogness]

tensor([[ 2.0617, -1.4259]], device='cuda:0', grad_fn=<AddmmBackward>)


C:\Users\vinograd\Anaconda3_12\envs\pytorch_gpu\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor([[0.9703, 0.0297]], device='cuda:0', grad_fn=<SoftmaxBackward>)


In [8]:
# 000a290e4-1 dog
# 00a1f270a-1 cat
img = cv2.imread('input/petfinder-adoption-prediction/train_images/000a290e4-1.jpg', 1)
#print(img)
img = np.float32(cv2.resize(img, (224, 224))) / 255
inp = preprocess_image(img)
#tryim = load_image('input/petfinder-adoption-prediction/train_images/','86e1089a3')
net = model_ft#torch.load('resnet50_trained3.pth')
#print(net)

net.eval()
output = net(inp)
print(output) #print output from crossentropy score

sm = torch.nn.Softmax()
probabilities = sm(output) 
print(probabilities) #Converted to probabilities

tensor([[ 0.3620, -0.2394]], device='cuda:0', grad_fn=<AddmmBackward>)


C:\Users\vinograd\Anaconda3_12\envs\pytorch_gpu\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


tensor([[0.6460, 0.3540]], device='cuda:0', grad_fn=<SoftmaxBackward>)


In [9]:
train_petids = train_df['PetID'].values
train_petids

array(['86e1089a3', '6296e909a', '3422e4906', ..., 'd981b6395',
       'e4da1c9e4', 'a83d95ead'], dtype=object)

In [18]:
train_prob_catness ={}
train_prob_dogness ={}
for train_petid in tqdm(train_petids[:5]): 
    row = data_df.loc[data_df['PetID'] == train_petid, :] 
    photo_amt = row['PhotoAmt'].values[0] 
    if photo_amt>0 :#and adoption>=0: 
        i=1 
        img_name = str(train_petid)+'-'+str(i)+'.jpg'
        img = cv2.imread(root_dir+img_name, 1)
        inp = preprocess_image(img)
        output = net(inp) #crossentropy score
        sm = torch.nn.Softmax()
        probabilities = sm(output) 
        probabilities = probabilities.cpu().detach().numpy()
        print(probabilities)
        train_prob_catness[train_petid]=probabilities[0][0]
        train_prob_dogness[train_petid]=probabilities[0][1]

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

tensor([[ 5.0952, -4.2838]], device='cuda:0', grad_fn=<AddmmBackward>)


C:\Users\vinograd\Anaconda3_12\envs\pytorch_gpu\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[[9.9991548e-01 8.4479616e-05]]
tensor([[ 3.4180, -2.5629]], device='cuda:0', grad_fn=<AddmmBackward>)
[[0.9974796  0.00252035]]
tensor([[-4.1203,  5.3020]], device='cuda:0', grad_fn=<AddmmBackward>)
[[8.0895625e-05 9.9991906e-01]]
tensor([[-4.7693,  6.3188]], device='cuda:0', grad_fn=<AddmmBackward>)
[[1.5292559e-05 9.9998474e-01]]
tensor([[-4.4899,  6.3032]], device='cuda:0', grad_fn=<AddmmBackward>)
[[2.0540278e-05 9.9997950e-01]]


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 45.99it/s]


In [19]:
train_prob_dogness

{'86e1089a3': 8.4479616e-05,
 '6296e909a': 0.002520352,
 '3422e4906': 0.99991906,
 '5842f1ff5': 0.99998474,
 '850a43f90': 0.9999795}

In [20]:
train_df["prob_catness"] = train_df.PetID.map(train_prob_catness)
train_df["prob_dogness"] = train_df.PetID.map(train_prob_dogness)

In [21]:
train_df["prob_catness"]

0        0.999915
1        0.997480
2        0.000081
3        0.000015
4        0.000021
5             NaN
6             NaN
7             NaN
8             NaN
9             NaN
10            NaN
11            NaN
12            NaN
13            NaN
14            NaN
15            NaN
16            NaN
17            NaN
18            NaN
19            NaN
20            NaN
21            NaN
22            NaN
23            NaN
24            NaN
25            NaN
26            NaN
27            NaN
28            NaN
29            NaN
           ...   
14963         NaN
14964         NaN
14965         NaN
14966         NaN
14967         NaN
14968         NaN
14969         NaN
14970         NaN
14971         NaN
14972         NaN
14973         NaN
14974         NaN
14975         NaN
14976         NaN
14977         NaN
14978         NaN
14979         NaN
14980         NaN
14981         NaN
14982         NaN
14983         NaN
14984         NaN
14985         NaN
14986         NaN
14987     

In [22]:
test_petids = test_df['PetID'].values
test_prob_catness ={}
test_prob_dogness ={}
root_dir=r'./input/petfinder-adoption-prediction/test_images/'
for test_petid in tqdm(test_petids[:5]): 
    row = test_df.loc[test_df['PetID'] == test_petid, :] 
    photo_amt = row['PhotoAmt'].values[0] 
    if photo_amt>0 :#and adoption>=0: 
        i=1 
        img_name = str(test_petid)+'-'+str(i)+'.jpg'
        img = cv2.imread(root_dir+img_name, 1)
        inp = preprocess_image(img)
        output = net(inp) #crossentropy score
        sm = torch.nn.Softmax()
        probabilities = sm(output) 
        probabilities = probabilities.cpu().detach().numpy()
        print(probabilities)
        test_prob_catness[test_petid]=probabilities[0][0]
        test_prob_dogness[test_petid]=probabilities[0][1]
test_df["prob_catness"] = test_df.PetID.map(test_prob_catness)
test_df["prob_dogness"] = test_df.PetID.map(test_prob_dogness)
test_df["prob_catness"].head()

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]C:\Users\vinograd\Anaconda3_12\envs\pytorch_gpu\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


[[0.00962085 0.9903791 ]]
[[9.9986374e-01 1.3619923e-04]]
[[0.9986753  0.00132468]]
[[0.7673059  0.23269409]]
[[0.00327893 0.9967211 ]]


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 52.77it/s]


0    0.009621
1    0.999864
2    0.998675
3    0.767306
4    0.003279
Name: prob_catness, dtype: float64

## Extract Train Image Features

In [ ]:
model = model_ft

In [ ]:
extract_transform = image_transforms['valid']

In [ ]:
train_pids = train_df.PetID.values
input_tensor = torch.zeros(1, 3, 224, 224)

train_image_features = {}
for petid in tqdm(train_pids):
    train_img = f"../input/train_images/{petid}-1.jpg"
    if not os.path.exists(train_img): continue
    
    train_img = Image.open(train_img)
    train_img = extract_transform(train_img)
    input_tensor[0, :, :, :] = train_img
    input_tensor = input_tensor.cuda()
    model(input_tensor)
    train_image_features[petid] = image_features[0]
    image_features.clear()


In [ ]:
train_image_features = pd.DataFrame.from_dict(train_image_features, orient='index')
train_image_features.columns = [f'img_nn_feat{idx}' for idx in train_image_features.columns.values]
train_image_features = train_image_features.reset_index().rename(columns={'index':'PetID'})

In [ ]:
train_image_features.head()

In [ ]:
train_image_features.to_csv('train_image_features.csv', index=False)

## Extract Test Image Features

In [ ]:
test_pids = test_df.PetID.values
input_tensor = torch.zeros(1, 3, 224, 224)

test_image_features = {}
for petid in tqdm(test_pids):
    test_img = f"../input/test_images/{petid}-1.jpg"
    if not os.path.exists(test_img): continue
    
    test_img = Image.open(test_img)
    test_img = extract_transform(test_img)
    input_tensor[0, :, :, :] = test_img
    input_tensor = input_tensor.cuda()
    model(input_tensor)
    test_image_features[petid] = image_features[0]
    image_features.clear()


In [ ]:
test_image_features = pd.DataFrame.from_dict(test_image_features, orient='index')
test_image_features.columns = [f'img_nn_feat{idx}' for idx in test_image_features.columns.values]
test_image_features = test_image_features.reset_index().rename(columns={'index':'PetID'})

In [ ]:
test_image_features.head()

In [ ]:
test_image_features.to_csv('test_image_features.csv', index=False)

We save the features as a csv to disk, so others can link and join the data frame with their train.csv and test.csv